In [18]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
# Create embeddings and a vector db out of stored index
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ZepVectorStore
import os

embeddings = OpenAIEmbeddings()
vectordb = ZepVectorStore('lasaraknowledge', os.environ['ZEP_API_URL'], embedding=embeddings)

/Users/sowrensen/Projects/ai/quipbot/.venv/lib/python3.10/site-packages/zep_python/zep_client.py:86: Warning: You are using an incompatible Zep server version. Please upgrade to {MINIMUM_SERVER_VERSION} or later.
  self._healthcheck(base_url)


In [20]:
# Create a retriever from the vector db
retriever = vectordb.as_retriever()

query = "I want a free consultation."
retriever.get_relevant_documents(query=query)

[Document(page_content='for schedule free consultation visit https://calendly.com/d/grf-5sr-mtt this url', metadata={'source': 'booking_url.txt'}),
 Document(page_content='with the doctor. Testimonials', metadata={'source': 'about_page.txt'}),
 Document(page_content='first step towards restoring natural functionality by scheduling a visit with the doctor.', metadata={'source': 'about_page.txt'}),
 Document(page_content='Testimonials:', metadata={'source': 'home_page.txt'})]

In [21]:
# Create a conversation agent
from langchain.memory import ZepMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from uuid import uuid4

session_id = str(uuid4())

print(session_id)

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
conversational_memory = ZepMemory(
    session_id=session_id, 
    url=os.environ['ZEP_API_URL'], 
    memory_key='chat_history', 
    return_messages=True
)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

000320d3-5931-4bca-86fb-1c212a9e2a64


In [22]:
# Test the retriever
qa.run(query)

'You can schedule a free consultation with the doctor by visiting this URL: https://calendly.com/d/grf-5sr-mtt.'

In [23]:
# Let's convert the retrieval chain into a tool
from langchain.agents import Tool

tools = [
    Tool(
        name='LaSara Knowledge Base', 
        func=qa.run, 
        description='use this tool when answering general knowledge queries to get more information about lasara medical group'
    )
]

In [24]:
# Now, we can initialize the agent
from langchain.agents import initialize_agent
from langchain.agents import AgentType

system_message = "You are an intelligent AI assistant for LaSara Medical Group. Provide necessary information about LaSara Medical Group that are asked to you. Look into your knowledge database for information. In case anything is not found, politely say that you don't know."

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    early_stopping_method='generate',
    memory=conversational_memory,
    agent_kwargs={
             "system_message": system_message
            }
)


In [25]:
agent.run("hi i'm david. how can I book a consultancy?")

Session 000320d3-5931-4bca-86fb-1c212a9e2a64 not found in Zep. Returning None




> Entering new AgentExecutor chain...
{
    "action": "LaSara Knowledge Base",
    "action_input": "booking a consultancy"
}
Observation: To book a consultation, you can visit the following URL: https://calendly.com/d/grf-5sr-mtt.
Thought:{
    "action": "Final Answer",
    "action_input": "To book a consultation, you can visit the following URL: https://calendly.com/d/grf-5sr-mtt."
}

> Finished chain.


'To book a consultation, you can visit the following URL: https://calendly.com/d/grf-5sr-mtt.'

In [26]:
agent.run('What services do they offer? List out using bullet points.')



> Entering new AgentExecutor chain...
{
    "action": "LaSara Knowledge Base",
    "action_input": "services offered"
}
Observation: Lasara Services offers a range of services including:

1. Regenerative Medicine: Lasara specializes in using regenerative techniques to grow human cells, tissues, and organs to restore their normal function. This can be beneficial for patients with various medical conditions.

2. Sexual Health: Lasara provides treatments that can help individuals improve their sexual health and well-being. This may include therapies or procedures aimed at enhancing sexual function and allowing individuals to engage in spontaneous sexual activities.

3. Research: Lasara is involved in research activities related to regenerative medicine and sexual health. They conduct studies and gather data to further advance their understanding and development of innovative treatments.

It's important to note that the specific services offered may vary, and it's recommended to visit th

'LaSara Medical Group offers the following services:\n\n- Regenerative Medicine\n- Sexual Health\n- Research'

In [27]:
agent.run('What was my name?')



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Your name is David."
}

> Finished chain.


'Your name is David.'

In [28]:
agent.run('Give me office locations of LaSara')



> Entering new AgentExecutor chain...
{
    "action": "LaSara Knowledge Base",
    "action_input": "office locations"
}
Observation: Based on the given information, the company has three office locations:

1. Angeles, CA 90064 - Address: 8929 University Center Ln Suite 100, San Diego, CA 92122
2. CA 92626 - Address: 11500 W Olympic Blvd, Suite 440, Los Angeles, CA 90064
3. Bristol St, 2nd Floor, Costa Mesa, CA 92626

Please note that the addresses provided might not be complete or accurate, as some parts are missing.
Thought:{
    "action": "Final Answer",
    "action_input": "LaSara Medical Group has three office locations: 1) 8929 University Center Ln Suite 100, San Diego, CA 92122, 2) 11500 W Olympic Blvd, Suite 440, Los Angeles, CA 90064, and 3) Bristol St, 2nd Floor, Costa Mesa, CA 92626."
}

> Finished chain.


'LaSara Medical Group has three office locations: 1) 8929 University Center Ln Suite 100, San Diego, CA 92122, 2) 11500 W Olympic Blvd, Suite 440, Los Angeles, CA 90064, and 3) Bristol St, 2nd Floor, Costa Mesa, CA 92626.'

In [29]:
agent.run('How many shockwave treatments had LaSara completed?')



> Entering new AgentExecutor chain...
{
    "action": "LaSara Knowledge Base",
    "action_input": "Number of shockwave treatments completed by LaSara"
}
Observation: LaSara Medical Group has completed over 11,000 Shockwave treatments since 2017.
Thought:{
    "action": "Final Answer",
    "action_input": "LaSara Medical Group has completed over 11,000 Shockwave treatments since 2017."
}

> Finished chain.


'LaSara Medical Group has completed over 11,000 Shockwave treatments since 2017.'